In [1]:
import gurobipy as gp

def is_interval(profile, print_intervals = False):
  # profile - list of lists of approved candidates
    n = len(profile)
    m = max([max(v) for v in profile if len(v)>0]) + 1


    Mod = gp.Model()
    Mod.params.OutputFlag = 0
    # we assume that all intervals are contained in positive half-space
    # and each interval is of lenght at most 1
    v_mid = {}
    v_rad = {}
    for v in range(n):
        v_mid[v] = Mod.addVar(vtype=gp.GRB.CONTINUOUS)
        v_rad[v] = Mod.addVar(vtype=gp.GRB.CONTINUOUS)
        Mod.addConstr(v_rad[v] >= 0.5)
        Mod.addConstr(v_mid[v] - v_rad[v] >= 0)
    
    c_mid = {}
    c_rad = {}
    for c in range(m):
        c_mid[c] = Mod.addVar(vtype=gp.GRB.CONTINUOUS)
        c_rad[c] = Mod.addVar(vtype=gp.GRB.CONTINUOUS)
        Mod.addConstr(c_rad[c] >= 0.5)
        Mod.addConstr(c_mid[c] - c_rad[c] >= 0)

    v_before_c = {} # binary variable checking if v is before c
    # we need them only for v, c pairs such that v does not approve c

    
    completed_variables = []
    for v in range(n):
        for c in range(m):
            if c in profile[v]:
                Mod.addConstr(v_mid[v] - c_mid[c] <= v_rad[v] + c_rad[c])
                Mod.addConstr(c_mid[c] - v_mid[v] <= v_rad[v] + c_rad[c])
            else:
                completed = Mod.addVar(vtype=gp.GRB.BINARY)
                completed_variables.append(completed)
                # if completed is 1, enforce the above constraints
                Mod.addConstr(v_mid[v] - c_mid[c] <= v_rad[v] + c_rad[c] + 1000 * (1 - completed))
                Mod.addConstr(c_mid[c] - v_mid[v] <= v_rad[v] + c_rad[c] + 1000 * (1 - completed))
                # if completed is 0, enforce the below constraints
                v_before_c[(v,c)] = Mod.addVar(vtype=gp.GRB.BINARY)
                Mod.addConstr(v_mid[v] - c_mid[c] + 1000 * v_before_c[(v,c)] + 1000 * completed >=
                            v_rad[v] + c_rad[c] + 0.05)
                Mod.addConstr(c_mid[c] - v_mid[v] + 1000 * (1 - v_before_c[(v,c)]) + 1000 * completed >=
                            v_rad[v] + c_rad[c] + 0.05)

    Mod.params.TimeLimit = 60
    Mod.setObjective(gp.quicksum(completed_variables), gp.GRB.MINIMIZE)
    

    """
    for v in range(n):
        for c in range(m):
            if c in profile[v]:
                Mod.addConstr(v_mid[v] - c_mid[c] <= v_rad[v] + c_rad[c])
                Mod.addConstr(c_mid[c] - v_mid[v] <= v_rad[v] + c_rad[c])
            else:
                v_before_c[(v,c)] = Mod.addVar(vtype=gp.GRB.BINARY)
                Mod.addConstr(v_mid[v] - c_mid[c] + 1000 * v_before_c[(v,c)] >=
                            v_rad[v] + c_rad[c] + 0.05)
                Mod.addConstr(c_mid[c] - v_mid[v] + 1000 * (1 - v_before_c[(v,c)]) >=
                            v_rad[v] + c_rad[c] + 0.05)
                # 0.05 added to fix problems with rounding

    # here, we minimize the sum of lengths of all intervals,
    # but we can use other objectives
    Mod.setObjective(gp.quicksum(v_rad[v] for v in range(n)) + 
                    gp.quicksum(c_rad[c] for c in range(m)), gp.GRB.MINIMIZE)

    """
    # here, we minimize the sum of lengths of all intervals,
    # but we can use other objectives
    Mod.optimize()
    # print ("Sum of completed variables:", sum([completed_variables[i].x for i in range(len(completed_variables))]))
    if Mod.SolCount > 0:
        return True, sum([completed_variables[i].x for i in range(len(completed_variables))])
    else:
        return False,0
        # print sum of completed_variables:
        if print_intervals:
            for v in range(n):
                print ('v' + str(v) + ": [" + str(round(v_mid[v].x - v_rad[v].x,4)) + 
                    " - " + str(round(v_mid[v].x + v_rad[v].x,4)) + "]")
            for c in range(m):
                print ('c' + str(c) + ": [" + str(round(c_mid[c].x - c_rad[c].x,4)) +
                    " - " + str(round(c_mid[c].x + c_rad[c].x,4)) + "]")
        return True
    
    

profile = [[0,1],[1,2],[2,0]]

print (is_interval(profile, True))

Set parameter Username
Academic license - for non-commercial use only - expires 2024-06-14
(True, 1.0)


In [27]:
import pandas as pd 
import numpy as np
import axisrules as axis
df = pd.read_csv('../../VoteData/Datasets/France 2022/Dynata/votes/approval.csv', index_col=0)
approval = df.to_numpy()
candidates = list(df.columns)
n_voters = 10
n_cand = 9
# random candidates 
# selected = np.random.choice(len(candidates), n_cand, replace=False)
# selected_voters = np.random.choice(len(approval), n_voters, replace=False)
# approval = approval[selected_voters][:,selected]
# candidates = [candidates[s] for s in selected]

In [24]:
import numpy as np 

def matrix2approval(profile):
    approval = []
    for i in range(profile.shape[0]):
        approval.append(list(np.where(profile[i,:]==1)[0]))
    return approval

votes = (matrix2approval(approval))

In [28]:
tot_bc = 0
tot_int = 0

from tqdm import tqdm 
c = 0

for _ in tqdm(range(100)):
    # random candidates 
    selected = np.random.choice(len(candidates), n_cand, replace=False)
    selected_voters = np.random.choice(len(approval), n_voters, replace=False)
    approval_i = approval[selected_voters][:,selected]
    candidates_i = [candidates[s] for s in selected]
    votes = (matrix2approval(approval_i))
    rule = axis.BallotCompletion(approval_i)
    res = rule.bruteforce()
    finished, score = is_interval(votes, False)
    if finished:
        c += 1
        tot_bc += rule.get_score(res[0][0])
        tot_int += score


 23%|██▎       | 23/100 [00:13<00:09,  8.31it/s]

In [26]:
print(tot_bc/100)
print(tot_int/100)
print("Under 1 minute:", c)

1.37
0.91
Under 1 minute: 100
